In [1]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
import random
import cx_Oracle
import re
from selenium.webdriver.chrome.service import Service
from openpyxl import Workbook
from selenium.common.exceptions import ElementNotInteractableException ## 추가됨
from selenium.common.exceptions import NoSuchElementException ## 추가됨
import sys ## 추가됨
import pandas as pd
import numpy as np
from konlpy.tag import Okt

In [2]:
#sql문으로 db에서 값 가져오기
def make_df(sql):
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    sql = sql
    cs.execute(sql)
    row = cs.fetchall()
    colname = cs.description
    col = []
    for i in colname:
        col.append(i[0])
    df =  pd.DataFrame(row,columns = col)
    return df

In [3]:
# okt를 통한 명사 분석 = 키워드 추출
def keyword_okt_noun(column):
    okt = Okt()
    arr = np.array(column)
    temp = []
    for i in range(len(arr)):
        temp.append(okt.nouns(arr[i]))
    noun = sum(temp,[])
    df = pd.DataFrame({'Noun':noun})
    noun_set = set(noun)
    noun_kind = list(noun_set)
    num=[]
    j=0
    for i in range(len(noun_kind)):
        num.append(len(df[df['Noun']==noun_kind[j]]))
        j=j+1
    #데이터프레임 생성 및 정렬
    df1 = pd.DataFrame({'Noun':noun_kind,'count':num})
    df1s = df1.sort_values('count',ascending=False)
    return df1s

In [4]:
def mk_nc_insert(mk_nc_df):
    df = mk_nc_df
    NOUN = list(df['Noun'])
    COUNT = list(df['count'])
    MAIN_K = select_main()[0][0] 
    
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    print('DB connect 성공!!')
    cs = conn.cursor()
    sql = "INSERT INTO MAIN_NOUN_COUNT (MAIN_K, NOUN, COUNT) VALUES(:1, :2, :3)"
    
    for i in range(len(df)):
        cs.execute(sql, (MAIN_K, NOUN[i], COUNT[i]))
        
    print('INSERT 성공!!')
    cs.close()
    conn.commit()
    conn.close()

In [5]:
#메인 키워드 가져오기.
def select_main():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    select_main_keyword= 'select MAIN_K from MAIN_KEYWORD'
    cs.execute(select_main_keyword)
    
    main_keyword = cs.fetchall()
    cs.close()
    conn.close()
    return main_keyword

In [ ]:
def word_cloud():
    

In [6]:
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient-basic-windows.x64-21.9.0.0.0dbru\instantclient_21_9")

In [7]:
if __name__ == '__main__':
    MK_DF = make_df('SELECT * FROM MAIN_TITLE WHERE TITLE IS NOT NULL')
    MK_NC = keyword_okt_noun(MK_DF['TITLE']).head(20)
    print(MK_NC)
    mk_nc_insert(MK_NC)

     Noun  count
93     은행     12
13     위기      7
83     미국      4
14      돈      3
151    금리      3
107    진짜      3
45     경제      3
130  한국은행      3
22    퍼스트      2
39     대출      2
141  리퍼블릭      2
85     투자      2
7      동결      2
86     지역      2
5      한국      2
115     문      1
110   대장동      1
106     법      1
108    제보      1
109     버      1
DB connect 성공!!
INSERT 성공!!
